In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import rebound
import reboundx

In [2]:
migration = "UNMigration" #"NoMigration" "UNMigration"
chemdist = "binary" #"binary" "linear" "no_overlap"
galactictides = "off" #"on" "off"
stellarperturb = "off" #"on" "off"

In [3]:
#Set locations for comets
def comets_loc(inner_lim, outer_lim, n_comets):
    comets = randomstate.uniform(low = inner_lim, high = outer_lim , size = n_comets)
    return comets

In [ ]:
#Set specific random seed for each cluster node sim, used to replicate for later
seedinput = 55 #2 #13 #55

In [4]:
randomstate = np.random.RandomState(seedinput)
#np.random.seed(seed=seedinput)
filenum = seedinput

n_comets = 100
#Set timesteps (tau = planetary migration timescale recalculation, enc = stellar encounter samples)
tau_timestep_num = 100
enc_timestep_num = 2 #449901 #always one more than (tmax - t_ce) / desired interval bewteeen steps
#Set interval at which rebound simulation snapshot is taken
sim_archive_interval = 5e4 #5e5

#Integrator used by simulation
sim_integrator = "mercurius" #"ias15" "whfast" "mercurius"

#Set distance (in terms of Hill Radii) at which integrator switches to IAS15 to handle close encounters (for Mercurius)
RcriticalMerc = 6.

#Set distance (in AU) at which objects are removed from the sim
object_exit_distance = 1e99 #206266. is 1 pc #1e99 for no removals

#tmax is the total simulation time
#T_ce is the time at which the planetary bodies stop moving (but the comets will move)
tmax = 4.5e9 #1.2e6 #4.5e9
T_ce = 1e6

#Set the semi-major axis and eccentricity timescales for Uranus and Neptune only
if migration == "UNMigration":
    a_i_list = [5.2044, 9.5826, 17.5, 27.5]
    e_i_list = [0.0489, 0.0565, 0.2, 0.3]
elif migration == "NoMigration":
    a_i_list = [5.2044, 9.5826, 19.2184, 30.1104]
    e_i_list = [0.0489, 0.0565, 0.0463, 0.00946]

a_f_list = [5.2044, 9.5826, 19.2184, 30.1104]
e_f_list = [0.0489, 0.0565, 0.0463, 0.00946]

In [5]:
if sim_integrator == "ias15":
    integrator_filelabel = "_IAS15_"
elif sim_integrator == "mercurius":
    integrator_filelabel = "_Rcrit"+str(RcriticalMerc)+"_"
    
if galactictides == "on":
    tides_filelabel = "_Tides"
else:
    tides_filelabel = ""
    
if stellarperturb == "on":
    perturb_filelabel = "_Perturb"
else:
    perturb_filelabel = ""

In [6]:
def simulation_load():
    #load Saved Simulation archive
    sa = rebound.SimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                                   tides_filelabel+perturb_filelabel+".bin")
    print("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                                   tides_filelabel+perturb_filelabel+".bin")
    print("Number of snapshots: %d" % len(sa))
    print("Time of first and last snapshot: %.1f, %.1f" % (sa.tmin, sa.tmax))
    for i in range(0,len(sa)):
        sim_saved = sa[i]
        print(sim_saved.t) 
        orbits_test = sim_saved.calculate_orbits()
        if len(orbits_test) == 0:
            print("Empty Simulation Found : Snapshot ",i)
            break

In [7]:
simulation_load()

sim_save_UNMigration_55_Rcrit6.0_binarychem.bin
Number of snapshots: 9001
Time of first and last snapshot: 0.0, 4500000000.0
0.0
500000.0
1000000.0
1500000.207678828
2000000.1188420246
2500000.029688916
3000000.2369942283
3500000.147764839
4000000.05853545
4500000.265840762
5000000.176611373
5500000.087381984
6000000.294687296
6500000.205457907
7000000.116228518
7500000.026999129
8000000.234304441
Empty Simulation Found : Snapshot  16


/home/Michael/anaconda3/lib/python3.6/site-packages/rebound/simulationarchive.py:127: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)
